In [42]:
import numpy as np
import open3d as o3d
import nibabel as nib
import pyvista
from skimage.measure import marching_cubes
import numpy as np
import time

### Get mesh via marching cube and visualize

In [34]:
# IDs of MRI used for experiments
ids_path = "/vol/space/projects/ukbb/projects/silhouette/eids_filtered.npy"
ids = np.load(ids_path)
ids

array([1000071, 1000180, 1000456, ..., 6023419, 6023786, 6023955])

In [35]:
# masked MRI example for visualization
mri_path = "/vol/space/projects/ukbb/projects/silhouette/labels/v1/3646767/body_mask.nii.gz"

In [36]:
body_segment = nib.load(mri_path)
body_segment_data = body_segment.get_fdata()
# create mesh from masked MRI image
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1) 
verts = verts/np.array(body_segment_data.shape) 
edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)

In [37]:
faces.shape

(511206, 3)

In [38]:
verts.shape

(255689, 3)

In [39]:
edges.shape

(1022412, 2)

In [40]:
# visualize original mesh
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

# uncomment out the following 2 lines for color-rendered visualization
# original_mesh_paint = np.asarray([0,200,220])/255.0
# mesh.paint_uniform_color(original_mesh_paint)

mesh.compute_vertex_normals()
mesh.compute_convex_hull()
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True,mesh_show_wireframe=False)


[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


### decimate mesh and visualize

In [41]:
# decimate mesh according to number of faces
decimation_level = 25000 # how many faces left after decimation
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))


start = time.time()
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, decimation_level)
end = time.time()

print("The time of execution of above program is :",(end-start) , "s")
print("The time of execution of above program is :",(end-start) * 10**3, "ms")

# visualize 

# uncomment out the following 2 lines for color-rendered visualization
# decimated_mesh_paint = np.asarray([230,200,110])/255.0
# decimated_mesh.paint_uniform_color(decimated_mesh_paint)
decimated_mesh.compute_vertex_normals()
decimated_mesh.compute_convex_hull()
o3d.visualization.draw_geometries([decimated_mesh],mesh_show_back_face=True,mesh_show_wireframe=False)

The time of execution of above program is : 7.601374387741089 s
The time of execution of above program is : 7601.374387741089 ms
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


### Visualization with different color rendering